In [1]:
import re
import pandas as pd

In [47]:
#import csv

food_desert = pd.read_csv('data/FoodAccessResearchAtlasData2019.csv', low_memory=False)

food_desert.head()

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga County,1,1912,693,0,0.0,0.00,0,...,221.0,1622.0,217.0,14.0,0.0,14.0,45.0,44.0,6.0,102.0
1,1001020200,Alabama,Autauga County,1,2170,743,0,181.0,8.34,1,...,214.0,888.0,1217.0,5.0,0.0,5.0,55.0,75.0,89.0,156.0
2,1001020300,Alabama,Autauga County,1,3373,1256,0,0.0,0.00,0,...,439.0,2576.0,647.0,17.0,5.0,11.0,117.0,87.0,99.0,172.0
3,1001020400,Alabama,Autauga County,1,4386,1722,0,0.0,0.00,0,...,904.0,4086.0,193.0,18.0,4.0,11.0,74.0,85.0,21.0,98.0
4,1001020500,Alabama,Autauga County,1,10766,4082,0,181.0,1.68,0,...,1126.0,8666.0,1437.0,296.0,9.0,48.0,310.0,355.0,230.0,339.0


In [48]:
food_desert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72531 entries, 0 to 72530
Columns: 147 entries, CensusTract to TractSNAP
dtypes: float64(126), int64(19), object(2)
memory usage: 81.3+ MB


In [49]:
#only keep important rows (kids 1 mile+ or 10 miles+ from supermarket), population, state, county, urban

food_desert = food_desert[['Pop2010', 'State','Urban','County','lakids1','lakids10',]]
food_desert

,Pop2010,State,Urban,County,lakids1,lakids10
0,1912,Alabama,1,Autauga County,504.0,NaN
1,2170,Alabama,1,Autauga County,406.0,NaN
2,3373,Alabama,1,Autauga County,416.0,NaN
3,4386,Alabama,1,Autauga County,346.0,NaN
4,10766,Alabama,1,Autauga County,715.0,NaN
...,...,...,...,...,...,...
72526,3326,Wyoming,0,Washakie County,716.0,177.0
72527,2665,Wyoming,1,Washakie County,NaN,NaN
72528,2542,Wyoming,1,Washakie County,30.0,NaN
72529,3314,Wyoming,0,Weston County,394.0,143.0


In [50]:
food_desert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72531 entries, 0 to 72530
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pop2010   72531 non-null  int64  
 1   State     72531 non-null  object 
 2   Urban     72531 non-null  int64  
 3   County    72531 non-null  object 
 4   lakids1   52542 non-null  float64
 5   lakids10  7766 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 3.3+ MB


In [52]:
#group by county, state, and add 
food_desert_counties = food_desert.groupby(['County','State','Urban']).sum().reset_index()

In [53]:
food_desert_counties = food_desert_counties.sort_values(['State','County'])
food_desert_counties.head()

,County,State,Urban,Pop2010,lakids1,lakids10
183,Autauga County,Alabama,0,25405,6333.0,1199.0
184,Autauga County,Alabama,1,29166,3640.0,0.0
196,Baldwin County,Alabama,0,99521,20756.0,516.0
197,Baldwin County,Alabama,1,82744,9877.0,0.0
213,Barbour County,Alabama,0,20819,3460.0,791.0


In [54]:
#calculate percentage of population and create columns

food_desert_counties['1mile+_percent'] = round(food_desert_counties['lakids1']/food_desert_counties['Pop2010'],2)
food_desert_counties['10mile+_percent'] = round(food_desert_counties['lakids10']/food_desert_counties['Pop2010'],2)
food_desert_counties

,County,State,Urban,Pop2010,lakids1,lakids10,1mile+_percent,10mile+_percent
183,Autauga County,Alabama,0,25405,6333.0,1199.0,0.25,0.05
184,Autauga County,Alabama,1,29166,3640.0,0.0,0.12,0.00
196,Baldwin County,Alabama,0,99521,20756.0,516.0,0.21,0.01
197,Baldwin County,Alabama,1,82744,9877.0,0.0,0.12,0.00
213,Barbour County,Alabama,0,20819,3460.0,791.0,0.17,0.04
...,...,...,...,...,...,...,...,...
4735,Uinta County,Wyoming,0,21118,3755.0,195.0,0.18,0.01
4912,Washakie County,Wyoming,0,3326,716.0,177.0,0.22,0.05
4913,Washakie County,Wyoming,1,5207,30.0,0.0,0.01,0.00
5044,Weston County,Wyoming,0,3314,394.0,143.0,0.12,0.04


In [62]:
#save as csv

food_desert_counties.to_csv('data/food_desert_counties_urb_rur.csv')

In [66]:
#create one more without urban column, to get general percents, regardless of type of location. Not sure which I will use...

food_desert_counties_sans = food_desert.drop(columns='Urban')
food_desert_counties_sans

,Pop2010,State,County,lakids1,lakids10
0,1912,Alabama,Autauga County,504.0,NaN
1,2170,Alabama,Autauga County,406.0,NaN
2,3373,Alabama,Autauga County,416.0,NaN
3,4386,Alabama,Autauga County,346.0,NaN
4,10766,Alabama,Autauga County,715.0,NaN
...,...,...,...,...,...
72526,3326,Wyoming,Washakie County,716.0,177.0
72527,2665,Wyoming,Washakie County,NaN,NaN
72528,2542,Wyoming,Washakie County,30.0,NaN
72529,3314,Wyoming,Weston County,394.0,143.0


In [67]:
#group by county, state, and add 

food_desert_counties_sans = food_desert_counties_sans.groupby(['State','County']).sum().reset_index()
food_desert_counties_sans

,State,County,Pop2010,lakids1,lakids10
0,Alabama,Autauga County,54571,9973.0,1199.0
1,Alabama,Baldwin County,182265,30633.0,516.0
2,Alabama,Barbour County,27457,3701.0,791.0
3,Alabama,Bibb County,22915,4198.0,90.0
4,Alabama,Blount County,57322,12575.0,0.0
...,...,...,...,...,...
3137,Wyoming,Sweetwater County,43806,6280.0,656.0
3138,Wyoming,Teton County,21294,2853.0,321.0
3139,Wyoming,Uinta County,21118,3755.0,195.0
3140,Wyoming,Washakie County,8533,746.0,177.0


In [68]:
#new columns with percentages

food_desert_counties_sans['1mile+_percent'] = round(food_desert_counties_sans['lakids1']/food_desert_counties_sans['Pop2010'],2)
food_desert_counties_sans['10mile+_percent'] = round(food_desert_counties_sans['lakids10']/food_desert_counties_sans['Pop2010'],2)
food_desert_counties_sans

,State,County,Pop2010,lakids1,lakids10,1mile+_percent,10mile+_percent
0,Alabama,Autauga County,54571,9973.0,1199.0,0.18,0.02
1,Alabama,Baldwin County,182265,30633.0,516.0,0.17,0.00
2,Alabama,Barbour County,27457,3701.0,791.0,0.13,0.03
3,Alabama,Bibb County,22915,4198.0,90.0,0.18,0.00
4,Alabama,Blount County,57322,12575.0,0.0,0.22,0.00
...,...,...,...,...,...,...,...
3137,Wyoming,Sweetwater County,43806,6280.0,656.0,0.14,0.01
3138,Wyoming,Teton County,21294,2853.0,321.0,0.13,0.02
3139,Wyoming,Uinta County,21118,3755.0,195.0,0.18,0.01
3140,Wyoming,Washakie County,8533,746.0,177.0,0.09,0.02


In [69]:
#save as csv

food_desert_counties_sans.to_csv('data/food_desert_counties_sans.csv')